[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

# Aplicaciones autenticadas.

En muchos casos, el acceso a un recurso de una aplicación requiera que el suario cuente con las credenciales y los permisos correctos.

*Django* incluye recursos que permiten crear funciones de vista que requieran de autenticación.

## La función ```django.contrib.auth.decorators.login_required```.

Esta función de primer orden permite ser aplicada como un decorador a aquellas funciones de vista que requieran que el usuario se autenticado con la siguiente sintaxis.

```
@login_required
def <función de vista>(request):
    ...
    ...
```

## Ejemplo ilustrativo.

### El script ```src/22/template_views.py```.

El script ```src/22/template_views.py``` contiene el siguiente código, el cual restringirá el acceso a las *URLs* que contienen plantillas de la aplicación ```api/```.

* En caso de que el usuario no haya ingresado correctamente, será redirigido a ```accounts/login/``` y una vez que haya hecho su ingreso exitoso, regresará a la *URL* original.

```python
from .models import Alumno
from .forms import FormaAlumno
from django.http import HttpResponseRedirect, HttpResponse, JsonResponse
from django.shortcuts import render
from requests import post
from django.contrib.auth.decorators  import login_required 

campos = ('numero_de_cuenta', 'nombre', 'primer_apellido', 'segundo_apellido', 'carrera', 'semestre', 'promedio', 'al_corriente')


@login_required
def vista(request):
    lista = [[(campo, getattr(alumno, campo)) for campo in campos] for alumno in Alumno.objects.all()]
        return render(request, 'listado.html',{'lista': lista}) 


@login_required
def valida(request):
    lista = [[getattr(alumno, campo) for campo in campos] for alumno in Alumno.objects.all()]
    return render(request,'valida.html',{'lista': lista}) 


@login_required
def forma(request):
    if request.method == 'POST':
        forma = FormaAlumno(request.POST)
        if forma.is_valid():
            datos = request.POST.dict()
            datos.pop('csrfmiddlewaretoken')
            cuenta = datos.pop('numero_de_cuenta')
            if 'al_corriente' in datos:
                datos['al_corriente']=True
            else:
                datos['al_corriente']=False
            resultado = post('http://' + request.get_host() + '/api/{}'.format(cuenta), data=datos)
            if resultado.status_code == 200:
                return HttpResponse('<h1>¡Alta Exitosa!</h1>')    
            else: 
                return HttpResponse('<h1>Ocurrió un error en el alta.</h1>')

            
    else:
        forma = FormaAlumno()

    return render(request, 'forma.html', {'forma': forma})
```


* Se sustituirá al script ```tutorial/api/template_views.py``` con el script ```src/22/template_views.py```.

* Para las plataforma GNU/Linux y MacOS X.

In [ ]:
!cp src/22/template_views.py tutorial/api/template_views.py

* Para la plataforma Windows.

In [ ]:
!copy src\22\template_views.py tutorial\api\template_views.py

* La siguiente celda mostrará el resultado de la sustitución.

In [ ]:
%pycat tutorial/api/template_views.py

## Arranque desde una terminal.

* Desde una terminal diríjase al directorio ```tutorial```, en el cual se encuentra el script ```manage.py```.
* Ejecute el siguiente comando:


```
python manage.py runserver 0.0.0.0:8000
```

**Nota:** 
Es necesario que el firewall de su equipo esté configurado para transmitir desde el puerto ```8000```. 

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2020.</p>